In [1]:
import numpy as np
import argparse
from custom_function import *
from glob import iglob
from collections import defaultdict
import scipy.ndimage
import scipy.io as sio
import sys
import os
import csv
from scipy.ndimage.interpolation import shift

def my_read_bin(cur_inp_file, data_type, input_shape):
  A = np.fromfile(cur_inp_file, dtype = data_type)
  A[np.isnan(A)] = 0
  A = np.reshape(A, input_shape)
  A = np.transpose(A, [2, 1, 0])
  return A

def center_to_def_loc(SA_rec, cur_loc, offset):
  SA_rec_sliced = SA_rec[:,:,cur_loc[2]+offset]
  sh_x = SA_rec_sliced.shape[1]//2 - cur_loc[0]
  sh_y = SA_rec_sliced.shape[0]//2 - cur_loc[1]
  SA_rec_sh = shift(SA_rec_sliced, [sh_y, sh_x], prefilter=False)
  return SA_rec_sh

def read_patient_file(file_path):
    """
    Reads the patient selection file and returns the lists of diseased and healthy patients.

    Parameters:
    file_path (str): The path to the patient selection text file.

    Returns:
    Tuple[List[str], List[str]]: A tuple containing two lists: one for diseased patients and one for healthy patients.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    diseased_patients = []
    healthy_patients = []
    is_diseased = False
    is_healthy = False

    for line in lines:
        line = line.strip()
        if line == "Selected diseased patients:":
            is_diseased = True
            is_healthy = False
        elif line == "Selected healthy patients:":
            is_diseased = False
            is_healthy = True
        elif is_diseased:
            if line:
                diseased_patients.append(line)
        elif is_healthy:
            if line:
                healthy_patients.append(line)

    return diseased_patients, healthy_patients


def read_columns_separately(file_path):
    # Initialize empty lists to store each column's values
    first_column = []
    second_column = []
    third_column = []

    # Open the file and read its content
    with open(file_path, 'r') as file:
        lines = file.readlines()

        # Iterate over each line in the file
        for line in lines:
            # Split the line by the tab character (or other delimiter)
            columns = line.strip().split('\t')

            # Ensure the line has at least three columns
            if len(columns) >= 3:
                # Append each column value to the corresponding list
                first_column.append(columns[0])
                second_column.append(columns[1])
                third_column.append(columns[2])

    return first_column, second_column, third_column

pat_id_arr_fname = f'/data01/user-storage/y.zezhang/2024_subsample_project/document/observer_study_patient_list/pat_id_diseased_pilot_defect_selected.txt'
diseased_patients = np.loadtxt(pat_id_arr_fname, dtype = 'str', comments="#", delimiter=",", unpack=False)

pat_id_arr_fname = f'/data01/user-storage/y.zezhang/2024_subsample_project/document/observer_study_patient_list/pat_id_healthy_pilot.txt'
healthy_patients = np.loadtxt(pat_id_arr_fname, dtype = 'str', comments="#", delimiter=",", unpack=False)


isIO=0


/tmp/ipykernel_2110339/2000071060.py:11: DeprecationWarning: Please import `shift` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.interpolation import shift


In [2]:

# Lists to hold patient IDs and codes
diseased_patient_ids = []
def_type_arr= []

# Process each entry in the list
for entry in diseased_patients:
    parts = entry.split()
    diseased_patient_ids.append(parts[0])
    def_type_arr.append(parts[1])


In [4]:
def_type_arr

['da2160s250',
 'di2160s250',
 'da2130s250',
 'da2130s250',
 'di2130s250',
 'di2160s250',
 'da2160s250',
 'da2160s250',
 'da2130s250',
 'di2160s250',
 'di2160s250',
 'di2130s250',
 'di2160s250',
 'da2160s250',
 'di2130s250',
 'da2130s250',
 'da2130s250',
 'di2130s250',
 'da2160s250',
 'di2160s250',
 'di2130s250',
 'da2130s250',
 'di2130s250',
 'da2160s250',
 'da2160s250',
 'da2160s250',
 'di2130s250',
 'da2160s250',
 'di2160s250',
 'da2130s250',
 'di2160s250',
 'di2160s250',
 'da2160s250',
 'di2160s250',
 'da2130s250',
 'da2130s250',
 'di2130s250',
 'di2160s250',
 'da2130s250',
 'da2160s250']